In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandasql import sqldf

from decision_trees.config import data_path
from decision_trees.plotting_utils import *

sns.set_context("talk")

# settings
%load_ext autoreload
%autoreload 2
pysqldf = lambda q: sqldf(q, globals())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Load In Data

Load in data (cleaned if was necessary)

In [8]:
fpath = "raw/breast_cancer_raw.csv"
df = pd.read_csv(data_path / fpath)
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Regional Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


## 2. Describe data

Things to consider:
* datatypes: cat, float, int, datetime
* distributions of numerical data ~ predictor
* multi-variate distributions 

https://ieee-dataport.org/open-access/seer-breast-cancer-data

more info on 6th edition stages: https://seer.cancer.gov/seerstat/variables/seer/ajcc-stage/6th/

Columns:
'Age': age of px at dx,
'Race': White, Black, Other, Other-Unspecified, Unknown (Other=Asian, Pacific Islander, American Indian)
'Marital Status': Single, Married, Separated, Divorced, Widowed. at time of dx,
'T Stage': size and extent of the main tumor (bigger is worse). T1-T4,
'N Stage': number of nearby affected lymph nodes. N1-N3,
'6th Stage': what stage of cancer - 6th edition stage of cancer, increases from IIA->IIB->IIIA->IIIB->IIIC,
'differentiate': range from Well->Moderate->Poorly->Undifferentiated, where undifferentiated is the worst,
'Grade': range from 1-4 (numerical encoding of "differentiate" column),
'A Stage': Regional or Distant. Distant means more metastasized (worse),
'Tumor Size': exact size in mm,
'Estrogen Status': if the tumor is Positive or Negative for estrogen receptors (if cancer cells are estrogen positive, it means it has receptors for estrogen, meaning estrogen levels affect the growth of the tumor. determining which receptor type you're positive for (estrogen and/or progesterone) determines what treatment options you have),
'Progesterone Status': Positive or Negative to progesterone receptors, same concept as estrogen,
'Regional Node Examined': total number of regional lymph nodes that were removed and examined by the pathologist,
'Regional Node Positive': total number of regional lymph nodes that were examined and found to be positive,
'Survival Months': amount of time px has survived since dx (if px is alive, this number is the min number of months they've survived, could be longer),
'Status': Alive or Dead

In [36]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   differentiate           4024 non-null   object
 7   Grade                   4024 non-null   object
 8   A Stage                 4024 non-null   object
 9   Tumor Size              4024 non-null   int64 
 10  Estrogen Status         4024 non-null   object
 11  Progesterone Status     4024 non-null   object
 12  Regional Node Examined  4024 non-null   int64 
 13  Regional Node Positive  4024 non-null   int64 
 14  Survival Months         4024 non-null   int64 
 15  Stat

In [37]:
df.columns

Index(['Age', 'Race', 'Marital Status', 'T Stage', 'N Stage', '6th Stage',
       'differentiate', 'Grade', 'A Stage', 'Tumor Size', 'Estrogen Status',
       'Progesterone Status', 'Regional Node Examined',
       'Regional Node Positive', 'Survival Months', 'Status'],
      dtype='object')

### Column Rename

In [ ]:
bool_map = {'Positive':True, 'Negative':False, 'Alive':True, 'Dead':False, 'Regional':False, 'Distant':True}
df["has_estrogen"] = df.estrogen_status.map(bool_map)
df["has_progesterone"] = df.progesterone_status.map(bool_map)
df["is_alive"] = df.status.map(bool_map)
df["is_metastasized"] = df.a_stage.map(bool_map)

df[["estrogen_status", "has_estrogen", "progesterone_status", "has_progesterone", "a_stage", "is_metastasized", "status", "is_alive"]].head()

In [40]:
df["Regional Node Examined"]

0       24
1       14
2       14
3        2
4        3
        ..
4019     1
4020    14
4021    11
4022    11
4023     7
Name: Regional Node Examined, Length: 4024, dtype: int64

In [51]:
query = """ 

select
    Age as age,
    Race as race,
    "Marital Status" as marital_status,
    Cast(Replace("T Stage", "T", "") AS INT) as t_stage, 
    Cast(Replace("N Stage", "N", "") AS INT) as n_stage, 
    Case
        when "6th Stage" = "IIA" then 1
        when "6th Stage" = "IIB" then 2
        when "6th Stage" = "IIIA" then 3
        when "6th Stage" = "IIIB" then 4
        when "6th Stage" = "IIIC" then 5
    end as stage,
    Grade as grade,
    "A Stage" as a_stage,
    "A Stage" = "Regional" as is_metastasized,
    "Tumor Size" as tumor_size,
    "Estrogen Status" as estrogen_status,
    "Estrogen Status" = "Positive" as has_estrogen,
    "Progesterone Status" as progesterone_status,
    "Progesterone Status" = "Positive" as has_progesterone,
    "Regional Node Examined" as regional_node_examined,
    "Regional Node Positive" as regional_node_positive,
    ROUND(CAST("Regional Node Positive" AS float)/CAST("Regional Node Examined" AS float), 4) as regional_node_ratio,
    "Survival Months" as survival_months,
    "Status" as status,
    "Status" = "Alive" as is_alive

from df 
"""
new_df = pysqldf(query)

### Boolean Representation

In [52]:
fpath = "processed/breast_cancer_sql.csv"
new_df.to_csv(data_path / fpath, index=False)